In [2]:
pip install textblob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.5 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.3/626.3 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 6.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
import ast
import datetime as dt

## Read tweets

In [31]:
raw_df = pd.read_csv("../../resources/final_tweets_corrected_2024.csv", encoding="ISO-8859-1")

/Users/u1452118/.conda/envs/FINANCE6500/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: DtypeWarning: Columns (30,33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
raw_df["text"] = raw_df["text"].str.strip()
raw_df.head()

,created_at,text,conversation_id,edit_history_tweet_ids,possibly_sensitive,reply_settings,lang,author_id,id,entities.urls,...,in_reply_to_user_id,username,name,location,keyword,entities.mentions,attachments.poll_ids,attachments.media_source_tweet_id,entities.cashtags,withheld.country_codes
0,2024-02-12T01:52:00.000Z,Fireman at the Superbowl https://t.co/AjqmmHvDQl,1.756859e+18,['1756858660205015180'],False,everyone,en,5.208863e+08,1.756859e+18,"[{'start': 25, 'end': 48, 'url': 'https://t.co...",...,NaN,jerkinjerky,Spencer's Jerkin' Jerky,"Gainesville, FL",max thieriot bode donovan,NaN,NaN,NaN,NaN,NaN
1,2024-02-12T01:20:00.000Z,"Watch the #SuperBowl ads for #GhostsCBS, #Fire...",1.756851e+18,['1756850607573623005'],False,everyone,en,1.809078e+07,1.756851e+18,"[{'start': 86, 'end': 109, 'url': 'https://t.c...",...,NaN,GiveMeMyRemote,GiveMeMyRemote,Wherever the TV news is.,max thieriot bode donovan,NaN,NaN,NaN,NaN,NaN
2,2024-02-12T01:15:36.000Z,Watch the #Tracker and #FireCountry #SuperBowl...,1.756828e+18,['1756849502018064847'],False,everyone,en,1.454115e+07,1.756850e+18,"[{'start': 65, 'end': 88, 'url': 'https://t.co...",...,14541151.0,marisaroffman,Marisa Roffman,NaN,max thieriot bode donovan,NaN,NaN,NaN,NaN,NaN
3,2024-02-12T01:15:17.000Z,I can't believe Norman Bates brother is a fire...,1.756849e+18,['1756849420442681386'],False,everyone,en,1.030866e+08,1.756849e+18,NaN,...,NaN,geekmentality,Geek Mentality Mike,Massachusetts,max thieriot bode donovan,NaN,NaN,NaN,NaN,NaN
4,2024-02-12T02:01:18.000Z,ð @NFL &amp; @fifa may be occut but some of...,1.756861e+18,['1756861000693739724'],False,everyone,en,1.747784e+18,1.756861e+18,"[{'start': 247, 'end': 270, 'url': 'https://t....",...,NaN,ryanadamtexas,Ryan Adam Texas,"Dallas, Texas USA",ultra beer,"[{'start': 2, 'end': 6, 'username': 'NFL', 'id...",NaN,NaN,NaN,NaN


In [4]:
tweets_df = raw_df[['text', 'conversation_id', 'keyword']]

In [5]:
tweets_df.head()

,text,conversation_id,keyword
0,Fireman at the Superbowl https://t.co/AjqmmHvDQl,1.756859e+18,max thieriot bode donovan
1,"Watch the #SuperBowl ads for #GhostsCBS, #Fire...",1.756851e+18,max thieriot bode donovan
2,Watch the #Tracker and #FireCountry #SuperBowl...,1.756828e+18,max thieriot bode donovan
3,I can't believe Norman Bates brother is a fire...,1.756849e+18,max thieriot bode donovan
4,ð @NFL &amp; @fifa may be occut but some of...,1.756861e+18,ultra beer


In [6]:
def clean_tweet(tweet):
    '''
    Utility function to clean tweet text by removing links, special characters
    using simple regex statements.
    '''
    return ' '.join(re.sub("(@)|([^0-9A-Za-z, \t])|(\w+:\/\/\S+)", "", tweet).split())

In [7]:
def get_tweet_sentiment(tweet):
    '''
    Utility function to classify sentiment of passed tweet
    using textblob's sentiment method
    '''
    # create TextBlob object of passed tweet text
    analysis = TextBlob(tweet)
    # set sentiment
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

In [8]:
tweets_df['cleaned_text'] = tweets_df['text'].apply(clean_tweet)

/Users/u1452118/.conda/envs/FINANCE6500/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [9]:
tweets_df.head()

,text,conversation_id,keyword,cleaned_text
0,Fireman at the Superbowl https://t.co/AjqmmHvDQl,1.756859e+18,max thieriot bode donovan,Fireman at the Superbowl
1,"Watch the #SuperBowl ads for #GhostsCBS, #Fire...",1.756851e+18,max thieriot bode donovan,"Watch the SuperBowl ads for GhostsCBS, FireCou..."
2,Watch the #Tracker and #FireCountry #SuperBowl...,1.756828e+18,max thieriot bode donovan,Watch the Tracker and FireCountry SuperBowl ad...
3,I can't believe Norman Bates brother is a fire...,1.756849e+18,max thieriot bode donovan,I cant believe Norman Bates brother is a fire ...
4,ð @NFL &amp; @fifa may be occut but some of...,1.756861e+18,ultra beer,NFL amp fifa may be occut but some of the athl...


In [10]:
tweets_df['sentiment'] = tweets_df['cleaned_text'].apply(get_tweet_sentiment)

/Users/u1452118/.conda/envs/FINANCE6500/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [11]:
tweets_df.head()

,text,conversation_id,keyword,cleaned_text,sentiment
0,Fireman at the Superbowl https://t.co/AjqmmHvDQl,1.756859e+18,max thieriot bode donovan,Fireman at the Superbowl,neutral
1,"Watch the #SuperBowl ads for #GhostsCBS, #Fire...",1.756851e+18,max thieriot bode donovan,"Watch the SuperBowl ads for GhostsCBS, FireCou...",positive
2,Watch the #Tracker and #FireCountry #SuperBowl...,1.756828e+18,max thieriot bode donovan,Watch the Tracker and FireCountry SuperBowl ad...,neutral
3,I can't believe Norman Bates brother is a fire...,1.756849e+18,max thieriot bode donovan,I cant believe Norman Bates brother is a fire ...,neutral
4,ð @NFL &amp; @fifa may be occut but some of...,1.756861e+18,ultra beer,NFL amp fifa may be occut but some of the athl...,positive


In [17]:
tweets_df.to_csv("../../resources/tweet-brand.csv", index=True)

## Brand 

In [222]:
raw_df[['keyword']].drop_duplicates().sort_values('keyword')


,keyword
22,AllState
38,America First Credit Union
468,Apartments.com
103719,BMW
11476,Bass Pro Shops and Cabela's
...,...
30676,us navy
440,veozah
31476,"victoria beckem, davis bechem, usher"
35794,wicked


In [12]:
keywords_df = pd.read_csv("../../resources/keywords file 2024_students.csv")
keywords_df.head()

,brandname,keywords,keywords_orig
0,AllState,(allstate) OR (mayhem) OR (mayhemlikeme) OR (p...,"all state, mayhem, parking gate, mayhem, allst..."
1,America First Credit Union,(5dollarbill) OR (abelincoln) OR (americafirst...,"5 dollar bill, america first, teenager, five d..."
2,Apple Music,(aliciakeys) OR (applemusic) OR (halftimeshow)...,"apple music, ludacris, yeah!, alicia keys, ush..."
3,Apartments.com,(alienapartments) OR (alieninvasion) OR (alien...,"jeff goldblum, alien apartments, extraterrestr..."
4,Bass Pro Shops and Cabela's,(5perday) OR (americasbestoutdoorretailer) OR ...,"conservation, stand up to all hate, making mem..."


#### Clean the tweet for brands

/Users/u1452118/.conda/envs/FINANCE6500/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,brandname,keywords,keywords_orig,brandname_cleaned
0,AllState,(allstate) OR (mayhem) OR (mayhemlikeme) OR (p...,"all state, mayhem, parking gate, mayhem, allst...",allstate
1,America First Credit Union,(5dollarbill) OR (abelincoln) OR (americafirst...,"5 dollar bill, america first, teenager, five d...",america first credit union
2,Apple Music,(aliciakeys) OR (applemusic) OR (halftimeshow)...,"apple music, ludacris, yeah!, alicia keys, ush...",apple music
3,Apartments.com,(alienapartments) OR (alieninvasion) OR (alien...,"jeff goldblum, alien apartments, extraterrestr...",apartmentscom
4,Bass Pro Shops and Cabela's,(5perday) OR (americasbestoutdoorretailer) OR ...,"conservation, stand up to all hate, making mem...",bass pro shops and cabelas
...,...,...,...,...
122,veozah,(menopause) OR (morenotflash) OR (notflash),"menopause, not flash, more not flash",veozah
123,"victoria beckem, davis bechem, usher",(butter) OR (peanut) OR (worthremembering),"worth remembering, peanut, butter","victoria beckem, davis bechem, usher"
124,weekdays,(gamechanger) OR (goodmorningfootball) OR (nfl),"game changer, nfl, goodmorning football",weekdays
125,wicked,(arianagrande) OR (ariangrande) OR (ethanslade...,"ariana grande, ariana grande, arian grande, wi...",wicked


#### Identify on Brand

In [13]:
quarterBrand = pd.read_csv("../../resources/Quarter and Brand Result.csv")
master_brand_list = quarterBrand['brandname']

def find_keywords(tweet):

    for brand in master_brand_list:
        if (brand and brand != ' ') and brand in tweet:
            return [brand]

tweets_df['brand'] = tweets_df['cleaned_text'].apply(lambda x: find_keywords(x))
tweets_df.head(20)

/Users/u1452118/.conda/envs/FINANCE6500/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,text,conversation_id,keyword,cleaned_text,sentiment,brand
0,Fireman at the Superbowl https://t.co/AjqmmHvDQl,1.756859e+18,max thieriot bode donovan,Fireman at the Superbowl,neutral,None
1,"Watch the #SuperBowl ads for #GhostsCBS, #Fire...",1.756851e+18,max thieriot bode donovan,"Watch the SuperBowl ads for GhostsCBS, FireCou...",positive,[CBS]
2,Watch the #Tracker and #FireCountry #SuperBowl...,1.756828e+18,max thieriot bode donovan,Watch the Tracker and FireCountry SuperBowl ad...,neutral,[CBS]
3,I can't believe Norman Bates brother is a fire...,1.756849e+18,max thieriot bode donovan,I cant believe Norman Bates brother is a fire ...,neutral,None
4,ð @NFL &amp; @fifa may be occut but some of...,1.756861e+18,ultra beer,NFL amp fifa may be occut but some of the athl...,positive,[NFL]
5,"ð´#XTREMO| #LionelMessi, #DanMarino and #Jas...",1.756841e+18,ultra beer,"XTREMO LionelMessi, DanMarino and JasonSudeiki...",neutral,None
6,"#LionelMessi, Dan ""The Man"" Marino and Jason S...",1.756835e+18,ultra beer,"LionelMessi, Dan The Man Marino and Jason Sude...",positive,[Michelob ULTRA]
7,#whataretheodds #SuperBowl #pringels #LionelMe...,1.756835e+18,ultra beer,whataretheodds SuperBowl pringels LionelMessi ...,neutral,[lindor]
8,#michelob #LionelMessi #Deadpool3 #Wrexham #Su...,1.756834e+18,ultra beer,michelob LionelMessi Deadpool3 Wrexham SuperBowl,neutral,None
9,#Popeyes with #KenJeong. Okay.\n#Michelob with...,1.756828e+18,ultra beer,Popeyes with KenJeong OkayMichelob with Lionel...,neutral,[Popeyes]


#### Identify on Hashtag

In [78]:
hashtag_df = pd.read_csv('../../resources/hashtag.csv')
hashtag_df.head()

/Users/u1452118/.conda/envs/FINANCE6500/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: DtypeWarning: Columns (6,7,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,created_at,text,possibly_sensitive,reply_settings,lang,geo.place_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,...,hashtag28,hashtag29,hashtag30,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50
0,2/12/24 2:35,RT @MatKat83: Hey Nerds!ðŸ‘‹ðŸ¤“ #KingdomOfThe...,FALSE,everyone,en,NaN,3,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2/12/24 2:35,Hey Nerds!ðŸ‘‹ðŸ¤“ #KingdomOfThePlanetOfTheApe...,FALSE,everyone,en,NaN,3,0,8,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2/12/24 2:22,RT @weeklycut: ðŸš¨#KingdomOfThePlanetOfTheApe...,FALSE,everyone,en,NaN,8,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2/12/24 2:21,RT @WandasAttorney: Marvel Studios is literall...,FALSE,everyone,en,NaN,268,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2/12/24 2:19,RT @WandasAttorney: Marvel Studios is literall...,FALSE,everyone,en,NaN,268,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
hashtag_df['hashtag'] = hashtag_df.iloc[:20].apply(lambda row: [row[col] for col in hashtag_df.loc[:,'hashtag1':'hashtag30'].columns if not pd.isna(row[col])], axis=1)

tweets_df = tweets_df.iloc[:20].merge(hashtag_df.iloc[:20][['text', 'hashtag']], on = 'text', how = 'left')
tweets_df.head(20)

,text,conversation_id,cleaned_text,brand,hashtag
0,Fireman at the Superbowl https://t.co/AjqmmHvDQl,1756858660205015180,Fireman at the Superbowl,None,NaN
1,"Watch the #SuperBowl ads for #GhostsCBS, #Fire...",1756850607573623005,"Watch the SuperBowl ads for GhostsCBS, FireCou...",CBS,NaN
2,Watch the #Tracker and #FireCountry #SuperBowl...,1756828194647527707,Watch the Tracker and FireCountry SuperBowl ad...,CBS,NaN
3,I can't believe Norman Bates brother is a fire...,1756849420442681386,I cant believe Norman Bates brother is a fire ...,None,NaN
4,ð @NFL &amp; @fifa may be occut but some of...,1756861000693739724,NFL amp fifa may be occut but some of the athl...,NFL,NaN
5,"ð´#XTREMO| #LionelMessi, #DanMarino and #Jas...",1756840844651270213,"XTREMO LionelMessi, DanMarino and JasonSudeiki...",None,NaN
6,"#LionelMessi, Dan ""The Man"" Marino and Jason S...",1756835100057071637,"LionelMessi, Dan The Man Marino and Jason Sude...",Michelob ULTRA,NaN
7,#whataretheodds #SuperBowl #pringels #LionelMe...,1756834563093868614,whataretheodds SuperBowl pringels LionelMessi ...,lindor,NaN
8,#michelob #LionelMessi #Deadpool3 #Wrexham #Su...,1756833630674944179,michelob LionelMessi Deadpool3 Wrexham SuperBowl,None,NaN
9,#Popeyes with #KenJeong. Okay.\n#Michelob with...,1756828268622225492,Popeyes with KenJeong OkayMichelob with Lionel...,Popeyes,NaN


In [14]:
keywords_df['keywords_list'] = keywords_df['keywords_orig'].replace(r'^\s*$', np.nan, regex=True)
keywords_df['keywords_list'] = keywords_df['keywords_list'].apply(lambda x: clean_tweet(x))
keywords_df['keywords_list'] = keywords_df['keywords_list'].apply(lambda x: x.split(','))
keywords_df['keywords_list'] = keywords_df['keywords_list'].apply(lambda x: [word.strip() for word in x])

stop_words = ["streaming", "stream", "superbowl", "nfl", "ai", "0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"]
def remove_stop_words(keywords_list):
    return [word for word in keywords_list if word.lower() not in stop_words]

# Apply the function to the 'keywords_list' column
keywords_df['keywords_list'] = keywords_df['keywords_list'].apply(remove_stop_words)

keywords_df['keywords_list'][49]

['couch potatoes',
 'pluto tv commerical',
 'potato',
 'couch potatoes',
 'couchpotato',
 'coach potato',
 'couch potato farms',
 'couch potatoes',
 'couch potatoes',
 'couch potato',
 'free',
 'potato',
 'couch potato',
 'pluto tv',
 'pluto commerical',
 'stream now pay never',
 'plutotv',
 'the finest free range couch potatoes',
 'pluto tv',
 'america is raised on tv',
 'farmer',
 'romance',
 'tractor',
 'stream now',
 'easy and free',
 'couch potato',
 'farmer',
 'pluto',
 'farm',
 'tv binging as a couch potato',
 'the way tv is supposed to be',
 'pluto tv',
 'country raised on tv',
 'country',
 'farm',
 'pay never',
 'plutotv',
 'couch potatoes',
 'farm',
 'plutotv couch potato farmers',
 'plutotv',
 'stream now pay never']

In [15]:
def find_keywords(tweet_row):
    brand_list = []
    tweet = tweet_row['cleaned_text']

    if tweet_row['brand'] is None:
        #print(tweet)
        for index, row in keywords_df.iterrows():
            for keyword in row['keywords_list']:
                if (keyword and keyword != ' ') and keyword.lower() in tweet.lower() and row['brandname'] not in brand_list and tweet_row['brand'] is None:
                    #print(keyword+'<-->'+row['brandname'], index)
                    if row['brandname'] in master_brand_list.values:
                        brand_list += [row['brandname']]
    else:
        brand_list = tweet_row['brand']
                
    return brand_list

tweets_df['brand'] = tweets_df.apply(find_keywords, axis = 1)


/Users/u1452118/.conda/envs/FINANCE6500/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [16]:
tweets_df.head()

,text,conversation_id,keyword,cleaned_text,sentiment,brand
0,Fireman at the Superbowl https://t.co/AjqmmHvDQl,1.756859e+18,max thieriot bode donovan,Fireman at the Superbowl,neutral,[]
1,"Watch the #SuperBowl ads for #GhostsCBS, #Fire...",1.756851e+18,max thieriot bode donovan,"Watch the SuperBowl ads for GhostsCBS, FireCou...",positive,[CBS]
2,Watch the #Tracker and #FireCountry #SuperBowl...,1.756828e+18,max thieriot bode donovan,Watch the Tracker and FireCountry SuperBowl ad...,neutral,[CBS]
3,I can't believe Norman Bates brother is a fire...,1.756849e+18,max thieriot bode donovan,I cant believe Norman Bates brother is a fire ...,neutral,"[CBS, Pluto TV]"
4,ð @NFL &amp; @fifa may be occut but some of...,1.756861e+18,ultra beer,NFL amp fifa may be occut but some of the athl...,positive,[NFL]


In [158]:

def populate_brand(row):
    if not row['brand']:
        return row['keyword']
    else:
        return row['brand']


tweets_df['brand'] = tweets_df.apply(populate_brand, axis=1)

/Users/u1452118/.conda/envs/FINANCE6500/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


### Annotations - JSON

In [185]:
annotation_df = raw_df[['conversation_id', 'entities.annotations']]
annotation_df['entities.annotations'][3]

"[{'start': 16, 'end': 27, 'probability': 0.9743, 'type': 'Person', 'normalized_text': 'Norman Bates'}, {'start': 61, 'end': 71, 'probability': 0.6336, 'type': 'Other', 'normalized_text': 'FireCountry'}, {'start': 74, 'end': 82, 'probability': 0.8799, 'type': 'Other', 'normalized_text': 'Superbowl'}]"

#### Annotation example

[{'start': 16, 'end': 27, 'probability': 0.9743, 'type': 'Person', 'normalized_text': 'Norman Bates'}, {'start': 61, 'end': 71, 'probability': 0.6336, 'type': 'Other', 'normalized_text': 'FireCountry'}, {'start': 74, 'end': 82, 'probability': 0.8799, 'type': 'Other', 'normalized_text': 'Superbowl'}]


In [186]:

def extract_person(data):
    if pd.notna(data):
        parsed_data = ast.literal_eval(data)
        persons = [d['normalized_text'] for d in parsed_data if d.get('type') == 'Person']
        return persons if persons else pd.NA
    else:
        return pd.NA

annotation_df['Person'] = annotation_df['entities.annotations'].apply(extract_person)
annotation_df.head()

/Users/u1452118/.conda/envs/FINANCE6500/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,conversation_id,entities.annotations,Person
0,1.756859e+18,"[{'start': 15, 'end': 23, 'probability': 0.827...",<NA>
1,1.756851e+18,"[{'start': 11, 'end': 19, 'probability': 0.910...",<NA>
2,1.756828e+18,"[{'start': 24, 'end': 34, 'probability': 0.532...",<NA>
3,1.756849e+18,"[{'start': 16, 'end': 27, 'probability': 0.974...",[Norman Bates]
4,1.756861e+18,"[{'start': 145, 'end': 148, 'probability': 0.6...","[messi, cristiano, mahomes, patrickmahomes, Je..."


In [188]:
annotation_df.to_csv("../../resources/tweet-person.csv", index=True)

### Engagement Metrics

In [33]:
engagement_df = raw_df[['conversation_id', "public_metrics.retweet_count", "public_metrics.reply_count", "public_metrics.like_count", "public_metrics.quote_count", "public_metrics.bookmark_count", "public_metrics.impression_count", "referenced_tweets"]]
engagement_df["public_metrics.retweet_count"] *= 0.2
engagement_df["public_metrics.reply_count"] *= 0.2
engagement_df["public_metrics.like_count"] *= 0.1
engagement_df["public_metrics.quote_count"] *= 0.2
engagement_df["public_metrics.bookmark_count"] *= 0.2
engagement_df.head()

/Users/u1452118/.conda/envs/FINANCE6500/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,conversation_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.bookmark_count,public_metrics.impression_count,referenced_tweets
0,1.756859e+18,0.0,0.2,0.3,0.0,0.0,387,NaN
1,1.756851e+18,0.0,0.0,0.2,0.0,0.0,478,NaN
2,1.756828e+18,0.0,0.2,0.1,0.0,0.0,698,"[{'type': 'replied_to', 'id': '175683759591149..."
3,1.756849e+18,0.0,0.0,0.1,0.0,0.0,186,NaN
4,1.756861e+18,0.0,0.0,0.0,0.0,0.0,376,NaN


In [34]:
def extract_referenceType(data):
    if pd.notna(data):
        parsed_data = ast.literal_eval(data)
        referenceType = 'reply' if parsed_data[0].get('type') == 'replied_to' else 'retweet'
        return referenceType if referenceType else pd.NA
    else:
        return "original"

engagement_df['tweet_type'] = engagement_df['referenced_tweets'].apply(extract_referenceType)
engagement_df.head()

/Users/u1452118/.conda/envs/FINANCE6500/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,conversation_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.bookmark_count,public_metrics.impression_count,referenced_tweets,tweet_type
0,1.756859e+18,0.0,0.2,0.3,0.0,0.0,387,NaN,original
1,1.756851e+18,0.0,0.0,0.2,0.0,0.0,478,NaN,original
2,1.756828e+18,0.0,0.2,0.1,0.0,0.0,698,"[{'type': 'replied_to', 'id': '175683759591149...",reply
3,1.756849e+18,0.0,0.0,0.1,0.0,0.0,186,NaN,original
4,1.756861e+18,0.0,0.0,0.0,0.0,0.0,376,NaN,original


In [35]:
def calculateScore(row):
    score = 0
    if row['tweet_type'] == 'original':
        score = row["public_metrics.retweet_count" : "public_metrics.bookmark_count"].sum()
    else:
        score = row["public_metrics.reply_count" : "public_metrics.bookmark_count"].sum()
    return score

In [36]:
engagement_df['engagement_score'] = engagement_df.apply(calculateScore, axis = 1)
engagement_df = engagement_df[['conversation_id', 'tweet_type', 'engagement_score', 'public_metrics.impression_count']]
engagement_df.sort_values(by='engagement_score', ascending=False).head()


/Users/u1452118/.conda/envs/FINANCE6500/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,conversation_id,tweet_type,engagement_score,public_metrics.impression_count
31380,1.756879e+18,original,4797.7,434711
11174,1.756882e+18,original,4005.4,514773
127064,1.756857e+18,original,3053.1,1563648
69673,1.756855e+18,original,2787.8,1046387
54789,1.756845e+18,original,2457.3,591031


In [37]:
engagement_df.to_csv("../../resources/tweet-engagement-score.csv", index=True)

### Quarter Data

In [22]:
raw_quarter_df = raw_df[['conversation_id', 'created_at']]
raw_quarter_df.head()

,conversation_id,created_at
0,1.756859e+18,2024-02-12T01:52:00.000Z
1,1.756851e+18,2024-02-12T01:20:00.000Z
2,1.756828e+18,2024-02-12T01:15:36.000Z
3,1.756849e+18,2024-02-12T01:15:17.000Z
4,1.756861e+18,2024-02-12T02:01:18.000Z


In [23]:


raw_quarter_df['ET_created_at'] = pd.to_datetime(raw_quarter_df['created_at']) - dt.timedelta(hours=5)
 
def quarter_assign(df):
  if df['ET_created_at'] >= pd.to_datetime('2024-02-11 18:30:00', utc = True) and df['ET_created_at'] < pd.to_datetime('2024-02-11 19:18:00', utc = True):
    return 'Q1'
  elif df['ET_created_at'] >= pd.to_datetime('2024-02-11 19:18:00', utc = True) and df['ET_created_at'] < pd.to_datetime('2024-02-11 20:25:00', utc = True):
    return 'Q2'
  elif df['ET_created_at'] >= pd.to_datetime('2024-02-11 20:25:00', utc = True) and df['ET_created_at'] < pd.to_datetime('2024-02-11 20:47:00', utc = True):
    return 'HT'
  elif df['ET_created_at'] >= pd.to_datetime('2024-02-11 20:47:00', utc = True) and df['ET_created_at'] < pd.to_datetime('2024-02-11 21:37:00', utc = True):
    return 'Q3'
  elif df['ET_created_at'] >= pd.to_datetime('2024-02-11 21:37:00', utc = True) and df['ET_created_at'] < pd.to_datetime('2024-02-11 22:22:00', utc = True):
    return 'Q4'
  elif df['ET_created_at'] >= pd.to_datetime('2024-02-11 22:22:00', utc = True) and df['ET_created_at'] < pd.to_datetime('2024-02-11 22:47:00', utc = True):
    return 'OT'
 
raw_quarter_df['quarter'] = raw_quarter_df.apply(quarter_assign, axis = 1)

/Users/u1452118/.conda/envs/FINANCE6500/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/Users/u1452118/.conda/envs/FINANCE6500/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [24]:
raw_quarter_df = raw_quarter_df[['conversation_id', 'quarter']]
raw_quarter_df.head()

,conversation_id,quarter
0,1.756859e+18,Q3
1,1.756851e+18,Q2
2,1.756828e+18,Q2
3,1.756849e+18,Q2
4,1.756861e+18,Q3


### Combined

In [25]:
del raw_df

import gc
gc.collect()

0

In [26]:
engagement_df.to_csv("../../resources/tweet-engagement-score.csv", index=True)
raw_quarter_df.to_csv("../../resources/tweet-quarter.csv", index=True)

In [2]:
combined_df = pd.merge(tweets_df, engagement_df, on='conversation_id', how='inner')
combined_df = pd.merge(combined_df, raw_quarter_df, on='conversation_id', how='inner')
combined_df.head()

NameError: name 'pd' is not defined

### Top Brands

In [43]:
metrics_df = pd.read_csv('../../resources/tweet-metrics.csv')[['brand', 'engagement_score', 'sentiment', 'public_metrics.impression_count', 'quarter']]
metrics_df.head()

,brand,engagement_score,sentiment,public_metrics.impression_count,quarter
0,[],0.5,neutral,387,Q3
1,['CBS'],0.2,positive,478,Q2
2,['CBS'],0.3,neutral,698,Q2
3,"['CBS', 'Pluto TV']",0.1,neutral,186,Q2
4,['NFL'],0.0,positive,376,Q3


In [42]:
quarterBrand = pd.read_csv("../../resources/Quarter and Brand Result.csv")
master_brand_list = quarterBrand[['brandname']]
master_brand_list['score'] = 0
master_brand_list['positive'] = 0
master_brand_list['neutral'] = 0
master_brand_list['negative'] = 0
master_brand_list['impression'] = 0
master_brand_list['Q1_tweets'] = 0
master_brand_list['Q2_tweets'] = 0
master_brand_list['HT_tweets'] = 0
master_brand_list['Q3_tweets'] = 0
master_brand_list['Q4_tweets'] = 0
master_brand_list['OT_tweets'] = 0
master_brand_list = master_brand_list.set_index('brandname')
master_brand_list.head()

,score,positive,neutral,negative,impression,Q1_tweets,Q2_tweets,HT_tweets,Q3_tweets,Q4_tweets,OT_tweets
brandname,,,,,,,,,,,
AllState,0,0,0,0,0,0,0,0,0,0,0
America First Credit Union,0,0,0,0,0,0,0,0,0,0,0
CBS,0,0,0,0,0,0,0,0,0,0,0
E.L.F. Cosmotics,0,0,0,0,0,0,0,0,0,0,0
Etsy,0,0,0,0,0,0,0,0,0,0,0


score       0
positivs    0
neutral     0
negative    0
Name: CBS, dtype: int64

In [44]:
def parse_brand(row):
    brand_list = ast.literal_eval(row['brand'])
    for brand in brand_list:
        if brand in master_brand_list.index:
            master_brand_list.loc[brand, 'score'] += row['engagement_score']
            master_brand_list.loc[brand, 'impression'] += row['public_metrics.impression_count']
            sentiment = row['sentiment']
            if sentiment == 'positive':
                master_brand_list.loc[brand, 'positive'] += 1
            if sentiment == 'neutral':
                master_brand_list.loc[brand, 'neutral'] += 1
            if sentiment == 'negative':
                master_brand_list.loc[brand, 'negative'] += 1
                
            quarter = row['quarter']
            if quarter == 'Q1':
                master_brand_list.loc[brand, 'Q1_tweets'] += 1
            if quarter == 'Q2':
                master_brand_list.loc[brand, 'Q2_tweets'] += 1
            if quarter == 'HT':
                master_brand_list.loc[brand, 'HT_tweets'] += 1
            if quarter == 'Q3':
                master_brand_list.loc[brand, 'Q3_tweets'] += 1
            if quarter == 'Q4':
                master_brand_list.loc[brand, 'Q4_tweets'] += 1
            if quarter == 'OT':
                master_brand_list.loc[brand, 'OT_tweets'] += 1
            
        else:
            # If brand does not exist in master_brand_list, you may want to handle it
            print(f"Brand '{brand}' not found in master_brand_list")

metrics_df.apply(parse_brand, axis=1)     
master_brand_list

,score,positive,neutral,negative,impression,Q1_tweets,Q2_tweets,HT_tweets,Q3_tweets,Q4_tweets,OT_tweets
brandname,,,,,,,,,,,
AllState,5.1,4,13,1,5599,0,6,0,5,6,1
America First Credit Union,8.9,27,8,8,11552,1,4,12,2,7,17
CBS,7071.1,14272,3527,906,7541558,2938,8709,1241,1305,2034,2478
E.L.F. Cosmotics,6283.7,3961,554,252,17935930,54,1068,1879,1333,178,255
Etsy,543.7,286,544,124,728866,60,497,134,112,66,85
...,...,...,...,...,...,...,...,...,...,...,...
us navy,0.6,27,4,16,652,7,5,9,10,8,8
veozah,5.5,15,5,3,4704,20,1,2,0,0,0
"victoria beckem, davis bechem, usher",20.3,27,44,45,24559,10,33,12,25,15,21


In [45]:
master_brand_list.to_csv("../../resources/brand-metrics.csv", index=True)